<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия №3
</center>
<center>Автор материала: программист-исследователь Mail.ru Group, старший преподаватель <br>Факультета Компьютерных Наук ВШЭ Юрий Кашницкий

# <center> Домашнее задание № 8. Решение
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

### 1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.Ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://docs.google.com/forms/d/1VaxYXnmbpeP185qPk2_V_BzbeduVUVyTdLPQwSCxDGA/edit) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в Wiki [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://drive.google.com/file/d/1ZU4J3KhJDrHVMj48fROFcTsTZKorPGlG/view) эту выборку. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [1]:
PATH_TO_DATA = '/Users/y.kashnitsky/Documents/stackoverflow_10mln/'

In [1]:
!head -1 $PATH_TO_DATA/stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [2]:
%%time
!wc -l $PATH_TO_DATA/stackoverflow.10kk.tsv

 10000000 stackoverflow.10kk.tsv
CPU times: user 285 ms, sys: 121 ms, total: 406 ms
Wall time: 16.1 s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлом на входе и на выходе
  - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
  - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
  - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
  - из текста вопроса надо выкинуть двоеточия и вертикальные палки – в VW это спецсимволы.

In [8]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк

In [4]:
!python preprocess_solution.py stackoverflow.10kk.tsv stackoverflow.vw

10000000it [01:38, 101508.51it/s]
4389054 lines selected, 15 lines corrupted.


Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции – по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train_part.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов на для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [2]:
%%time
!split -l 1463018 $PATH_TO_DATA/stackoverflow.vw $PATH_TO_DATA/stackoverflow_

CPU times: user 497 ms, sys: 156 ms, total: 653 ms
Wall time: 33 s


In [3]:
!mv $PATH_TO_DATA/stackoverflow_aa $PATH_TO_DATA/stackoverflow_train.vw
!mv $PATH_TO_DATA/stackoverflow_ab $PATH_TO_DATA/stackoverflow_valid.vw
!mv $PATH_TO_DATA/stackoverflow_ac $PATH_TO_DATA/stackoverflow_test.vw

In [4]:
!wc -l $PATH_TO_DATA/stackoverflow_*.vw

 1463018 /Users/y.kashnitsky/Documents/stackoverflow_10mln//stackoverflow_test.vw
 1463018 /Users/y.kashnitsky/Documents/stackoverflow_10mln//stackoverflow_train.vw
 1463018 /Users/y.kashnitsky/Documents/stackoverflow_10mln//stackoverflow_valid.vw
 4389054 total


In [5]:
%%time
!cat $PATH_TO_DATA/stackoverflow_valid.vw \
    | cut -f 1 -d ' ' > $PATH_TO_DATA/stackoverflow_valid_labels.txt
!cat $PATH_TO_DATA/stackoverflow_test.vw \
    | cut -f 1 -d ' ' > $PATH_TO_DATA/stackoverflow_test_labels.txt

CPU times: user 402 ms, sys: 127 ms, total: 529 ms
Wall time: 31.9 s


Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: `loss_function=hinge`, `bit_precision`=28 и `seed`=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [6]:
def train_vw_model(train_vw_file, model_filename, num_classes=10,
                   ngram=1, loss_function='hinge',
                   bit_precision=28, passes=1,
                   seed=17, quiet=True):
    init_time = time()
    vw_call_string = ('vw --oaa {num_classes} {train_vw_file} ' + 
                      '-f {model_filename} -b {bit_precision} ' +
                      '--loss_function {loss_function} --random_seed {seed}').format(
                       num_classes=num_classes, train_vw_file=train_vw_file, 
                        loss_function=loss_function,
                       model_filename=model_filename, 
        bit_precision=bit_precision, seed=seed)
    if ngram > 1:
         vw_call_string += ' --ngram={}'.format(ngram)
            
    if passes > 1:
         vw_call_string += ' -k --passes={} --cache_file {}'.format(passes, 
                            model_filename.replace('.vw', '.cache'))
    if quiet:
        vw_call_string += ' --quiet'
    
    
    print(vw_call_string) 
    res = os.system(vw_call_string)
    print('Success. Elapsed: {} sec.'.format(round(time() - init_time, 2))
          if not res else 'Failed.')

In [7]:
def test_vw_model(model_filename, test_vw_file, prediction_filename,
                  true_labels, seed=17, quiet=True):
    init_time = time()
    vw_call_string = ('vw -t -i {model_filename} {test_vw_file} ' + 
                       '-p {prediction_filename} --random_seed {seed}').format(
                       model_filename=model_filename, test_vw_file=test_vw_file, 
                       prediction_filename=prediction_filename, seed=seed)
    if quiet:
        vw_call_string += ' --quiet'
        
    print(vw_call_string) 
    res = os.system(vw_call_string)
    
    if not res: # the call resulted OK
        vw_pred = np.loadtxt(prediction_filename)
        print("Accuracy: {}%. Elapsed: {} sec.".format(
            round(100 * accuracy_score(true_labels, vw_pred), 2), 
            round(time() - init_time, 2)))
    else:
        print('Failed.')

In [9]:
y_valid = np.loadtxt(os.path.join(PATH_TO_DATA,
                                  'stackoverflow_valid_labels.txt'))
y_test = np.loadtxt(os.path.join(PATH_TO_DATA,
                                 'stackoverflow_test_labels.txt'))

In [10]:
import itertools

In [11]:
for i, (ngram, passes) in tqdm(enumerate(itertools.product([1,2,3], 
                                                      [1,3,5]))):
    train_vw_model(os.path.join(PATH_TO_DATA, 'stackoverflow_train.vw'),
                   os.path.join(PATH_TO_DATA,
                                'vw_model{}_part.vw'.format(i)), 
                   ngram=ngram, passes=passes,
                   loss_function='hinge',
                   num_classes=10, bit_precision=28, 
                   seed=17, quiet=True)
    test_vw_model(model_filename=os.path.join(PATH_TO_DATA, 
                                              'vw_model{}_part.vw'.format(i)), 
              test_vw_file=os.path.join(PATH_TO_DATA,
                                        'stackoverflow_valid.vw'), 
              prediction_filename=os.path.join(PATH_TO_DATA,
                                               'vw_valid_pred{}.csv'.format(i)),
              true_labels=y_valid, seed=17)

0it [00:00, ?it/s]

vw --oaa 10 /Users/y.kashnitsky/Documents/stackoverflow_10mln/stackoverflow_train.vw -f /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model0_part.vw -b 28 --loss_function hinge --random_seed 17 --quiet
Success. Elapsed: 18.77 sec.
vw -t -i /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model0_part.vw /Users/y.kashnitsky/Documents/stackoverflow_10mln/stackoverflow_valid.vw -p /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_valid_pred0.csv --random_seed 17 --quiet


1it [00:47, 47.25s/it]

Accuracy: 91.67%. Elapsed: 28.48 sec.
vw --oaa 10 /Users/y.kashnitsky/Documents/stackoverflow_10mln/stackoverflow_train.vw -f /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model1_part.vw -b 28 --loss_function hinge --random_seed 17 -k --passes=3 --cache_file /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model1_part.cache --quiet
Success. Elapsed: 57.22 sec.
vw -t -i /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model1_part.vw /Users/y.kashnitsky/Documents/stackoverflow_10mln/stackoverflow_valid.vw -p /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_valid_pred1.csv --random_seed 17 --quiet


2it [02:14, 59.13s/it]

...

Accuracy: 93.0%. Elapsed: 78.18 sec.
vw --oaa 10 /Users/y.kashnitsky/Documents/stackoverflow_10mln/stackoverflow_train.vw -f /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model7_part.vw -b 28 --loss_function hinge --random_seed 17 --ngram=3 -k --passes=3 --cache_file /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model7_part.cache --quiet
Success. Elapsed: 404.11 sec.
vw -t -i /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model7_part.vw /Users/y.kashnitsky/Documents/stackoverflow_10mln/stackoverflow_valid.vw -p /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_valid_pred7.csv --random_seed 17 --quiet


8it [27:16, 283.21s/it]

Accuracy: 92.73%. Elapsed: 78.52 sec.
vw --oaa 10 /Users/y.kashnitsky/Documents/stackoverflow_10mln/stackoverflow_train.vw -f /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model8_part.vw -b 28 --loss_function hinge --random_seed 17 --ngram=3 -k --passes=5 --cache_file /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model8_part.cache --quiet
Success. Elapsed: 486.74 sec.
vw -t -i /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model8_part.vw /Users/y.kashnitsky/Documents/stackoverflow_10mln/stackoverflow_valid.vw -p /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_valid_pred8.csv --random_seed 17 --quiet


9it [36:42, 367.84s/it]

Accuracy: 92.81%. Elapsed: 78.59 sec.


<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- Биграммы и 1 проход по выборке **[+]**
- Униграммы и 3 прохода по выборке

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

In [12]:
test_vw_model(model_filename=os.path.join(PATH_TO_DATA,
                                          'vw_model3_part.vw'), 
              test_vw_file=os.path.join(PATH_TO_DATA,
                                        'stackoverflow_test.vw'), 
              prediction_filename=os.path.join(PATH_TO_DATA,
                                               'vw_test_pred3.csv'),
              true_labels=y_test, seed=17)

vw -t -i /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model3_part.vw /Users/y.kashnitsky/Documents/stackoverflow_10mln/stackoverflow_test.vw -p /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_test_pred3.csv --random_seed 17 --quiet
Accuracy: 93.25%. Elapsed: 55.19 sec.


<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках?
- На тестовой ниже примерно на 1%
- На тестовой ниже примерно на 1%
- Результаты почти одинаковы – отличаются меньше чем на 0.5% **[+]**

In [15]:
%%time
!cp $PATH_TO_DATA/stackoverflow_train.vw $PATH_TO_DATA/stackoverflow_train_valid.vw
!cat $PATH_TO_DATA/stackoverflow_valid.vw >> $PATH_TO_DATA/stackoverflow_train_valid.vw

CPU times: user 240 ms, sys: 93.1 ms, total: 333 ms
Wall time: 13.9 s


Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [16]:
train_vw_model(os.path.join(PATH_TO_DATA, 'stackoverflow_train_valid.vw'), 
                   os.path.join(PATH_TO_DATA, 'vw_model10.vw'), 
                   ngram=2, passes=1,
                   num_classes=10, bit_precision=28, 
                   seed=17, quiet=True)

vw --oaa 10 /Users/y.kashnitsky/Documents/stackoverflow_10mln/stackoverflow_train_valid.vw -f /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model10.vw -b 28 --loss_function hinge --random_seed 17 --ngram=2 --quiet
Success. Elapsed: 117.87 sec.


In [17]:
test_vw_model(model_filename=os.path.join(PATH_TO_DATA, 'vw_model10.vw'), 
              test_vw_file=os.path.join(PATH_TO_DATA,
                                        'stackoverflow_test.vw'), 
              prediction_filename=os.path.join(PATH_TO_DATA,
                                               'vw_test_pred10.csv'),
              true_labels=y_test, seed=17)

vw -t -i /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_model10.vw /Users/y.kashnitsky/Documents/stackoverflow_10mln/stackoverflow_test.vw -p /Users/y.kashnitsky/Documents/stackoverflow_10mln/vw_test_pred10.csv --random_seed 17 --quiet
Accuracy: 93.64%. Elapsed: 54.73 sec.


<font color='red'> Вопрос 3.</font> На сколько повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - 0.4% **[+]**
 - 0.8%
 - 1.2%